In [ ]:
path = "../../"

In [ ]:
import os
import torch
import pandas as pd
import numpy as np
from PIL import Image
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils, datasets
from torch.utils.data import random_split

In [ ]:
random_seed = 123 # set random seed
torch.manual_seed(random_seed)
torch.cuda.manual_seed(random_seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
np.random.seed(random_seed)

In [ ]:
train_root = os.path.join(path, "data/asl/train/")
test_root = os.path.join(path, "data/asl/test/")

In [ ]:
# following code running time is long, so I command it
# dataset = datasets.ImageFolder(root = train_root, transform=transforms.ToTensor())
# train_split = 0.8 # split train and validation as 8:2
# train_size = int(train_split * len(dataset))
# val_size = len(dataset) - train_size
# train_dataset, val_dataset = random_split(dataset, [train_size, val_size],
#                                       generator=torch.manual_seed(random_seed)) 
# imgs = torch.stack([img_t for img_t, _ in train_dataset], dim=3)
# mean = imgs.view(3, -1).mean(dim=1)
# std = imgs.view(3, -1).std(dim=1)
print("means is [0.5190, 0.4991, 0.5139], variances is [0.2283, 0.2557, 0.2639]")

## Preprocessing nomalization
### find the original train data (under the random seed 42) statistics including means and variances of 3 channels.

In [ ]:
dataset = datasets.ImageFolder(
    root = train_root, 
    transform = transforms.Compose([transforms.ToTensor(),
                                    transforms.Normalize((0.5190, 0.4991, 0.5139),
                                                         (0.2283, 0.2557, 0.2639))])
    )
batch_size = 16 
train_split = 0.8 # split train and validation as 8:2

train_size = int(train_split * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = random_split(dataset, [train_size, val_size]) 
# already set random seed in the beginning

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
validation_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=True)

In [ ]:
print(dataset.classes[:])
print(dataset.class_to_idx)

In [ ]:
train_dataset[0]

In [ ]:
import matplotlib.pyplot as plt

def show_batch(imgs_batched):
    """Show image for a batch of samples."""

    grid = utils.make_grid(imgs_batched)
    plt.imshow(grid.numpy().transpose((1, 2, 0)))
    plt.title('Batch from dataloader')

i_batch = 0
for img, label in train_loader:
    print(i_batch, img.size(), label.size())

    # observe 4th batch and stop.
    if i_batch == 3:
        plt.figure()
        show_batch(img)
        plt.axis('off')
        plt.ioff()
        plt.show()
        break
    i_batch += 1